In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch
from torch.nn import functional as F 
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import numpy as np
import os


c:\Users\vitta\Desktop\F5_Bootcamp\G2_Youtube\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("../data/youtoxic_english_processed.csv")
df = df[["clean_text", "IsToxic"]]
df.head()


,clean_text,IsToxic
0,people would take step back make case anyone e...,0
1,law enforcement trained shoot apprehend traine...,1
2,dont reckon black lives matter banners held wh...,1
3,large number people like police officers calle...,0
4,arab dude absolutely right shot extra time sho...,0


In [3]:
# Tokenización
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(clean_text):
    return tokenizer(
        clean_text,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )
tokenized_data = tokenize_data(df["clean_text"].tolist())



In [4]:
# Crear dataset
class ToxicDataset(Dataset):
    def __init__(self, tokenized_data, labels, augment=False):
        self.input_ids = tokenized_data["input_ids"]
        self.attention_mask = tokenized_data["attention_mask"]
        self.labels = torch.tensor(labels, dtype=torch.long)
        self.augment = augment
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx]
        }


In [5]:
# Función de entrenamiento y evaluación
def train_one_epoch(model, dataloader, optimizer, device, scheduler, class_weights):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    accumulation_steps = 2
    optimizer.zero_grad()
    
    progress_bar = tqdm(dataloader, desc="Training", total=len(dataloader))
    for i, batch in enumerate(progress_bar):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss
        weighted_loss = loss * class_weights[labels]
        loss = weighted_loss.mean()
        
        loss.backward()
        
        if (i + 1) % accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        total_loss += loss.item()
        predictions = torch.argmax(outputs.logits, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)
        
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}', 
            'accuracy': f'{correct / total:.4f}'
            })
        
    return total_loss / len(dataloader), correct / total

def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            
            total_loss += loss.item()
            
            predictions = torch.argmax(outputs.logits, dim=1)
            correct += (predictions == labels).sum().item()
            total += len(labels)
        if total == 0:
            return 0.0, 0.0
        
    return total_loss / len(dataloader), correct / total

        


In [6]:
# Entrenamiento
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Parámetros
batch_size = 16
num_epochs = 8
learning_rate = 2e-5
patience = 5
dropout_rate = 0.2
weight_decay = 0.01

# Calcular pesos de las clases
class_weights = compute_class_weight(
    'balanced', 
    classes=np.unique(df['IsToxic']),
    y=df['IsToxic']
)
class_weights = torch.tensor(
    class_weights, 
    dtype=torch.float
    ).to(device)

# Verificar la distribución de los datos
print("\nDistribución de clases:")
print(df['IsToxic'].value_counts())


train_idx, val_idx = train_test_split(
    np.arange(len(df)),
    test_size=0.2,
    stratify=df['IsToxic'],
    random_state=42
)

print(f"\nTamaño del conjunto de entrenamiento: {len(train_idx)}")
print(f"Tamaño del conjunto de validación: {len(val_idx)}")
    
# Verificar distribución de clases en cada split
train_dist = df.iloc[train_idx]['IsToxic'].value_counts()
val_dist = df.iloc[val_idx]['IsToxic'].value_counts()
print("\nDistribución de clases en entrenamiento:")
print(train_dist)
print("\nDistribución de clases en validación:")
print(val_dist)

# Ajustar batch_size basado en el tamaño de los datasets
actual_batch_size = min(
    batch_size,
    len(train_idx) // 10,
    len(val_idx)
    )
actual_batch_size = max(1, actual_batch_size)  # Asegurar que no sea 0
print(f"Batch size ajustado: {actual_batch_size}")
# Crear data set 
train_dataset = ToxicDataset({
    "input_ids": tokenized_data["input_ids"][train_idx],
    "attention_mask": tokenized_data["attention_mask"][train_idx],
    }, df["IsToxic"].iloc[train_idx].values, augment=True)

val_dataset = ToxicDataset({
    "input_ids": tokenized_data["input_ids"][val_idx],
    "attention_mask": tokenized_data["attention_mask"][val_idx],
    }, df["IsToxic"].iloc[val_idx].values, augment=False)
    
    # Verificar que los datasets no estén vacíos
if len(train_dataset) == 0 or len(val_dataset) == 0:
    print("Error: Dataset vacío")

# Crear dataloaders
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=actual_batch_size, 
    shuffle=True,
    drop_last=False
)
val_dataloader = DataLoader(
    val_dataset, 
    batch_size=actual_batch_size,
    shuffle=False,
    drop_last=False
)

# Inicializar modelo
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    hidden_dropout_prob=dropout_rate,
    attention_probs_dropout_prob=dropout_rate,
    classifier_dropout=dropout_rate,
    hidden_act="gelu"
    ).to(device)

# Optimizador
optimizer = AdamW(
    model.parameters(), 
    lr=learning_rate,
    weight_decay=weight_decay,
    eps=1e-8,
    betas=(0.9, 0.999)
    )
# Scheduler
num_training_steps = len(train_dataloader) * num_epochs
num_warmup_steps = num_training_steps // 10
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

#Early stopping setup
best_val_accuracy = 0
patience_counter = 0

# Training loop
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    
    # Train
    train_loss, train_accuracy = train_one_epoch(
        model, 
        train_dataloader, 
        optimizer, 
        device, 
        scheduler,
        class_weights
        )
    
    # Evaluate
    val_loss, val_accuracy = evaluate(model, val_dataloader, device)
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
    # Early stopping check
    if abs(train_accuracy - val_accuracy) > 0.1: # 10% de diferencia
        print("Warning: Posible overfitting detectado")
        if patience_counter == 0:
            # Reducir learning rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = param_group['lr'] * 0.5
            print(f"Learning rate reducida a {optimizer.param_groups[0]['lr']:.6f}")

    if val_accuracy > best_val_accuracy:
        current_overfitting = abs(train_accuracy - val_accuracy) * 100
        
        if current_overfitting <= 5:  # Solo guardamos si el overfitting es menor al 5%
            best_val_accuracy = val_accuracy
            best_train_accuracy = train_accuracy
            best_overfitting = current_overfitting
            patience_counter = 0
            
            # Guardar el mejor modelo
            if not os.path.exists('../models'):
                os.makedirs('../models')
            
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_accuracy': train_accuracy,
                'val_accuracy': val_accuracy,
                'overfitting': current_overfitting,
                'epoch': epoch
            }, '../models/toxic_comment_model.pt')
            
            print(f"\nGuardado nuevo mejor modelo:")
            print(f"Accuracy de validación: {val_accuracy:.4f}")
            print(f"Accuracy de entrenamiento: {train_accuracy:.4f}")
            print(f"Overfitting: {current_overfitting:.2f}%")
        else:
            print(f"No se guardó el modelo - Overfitting ({current_overfitting:.2f}%) > 5%")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

# Al final del entrenamiento (después del loop)
try:
    # Cargar el mejor modelo guardado
    checkpoint = torch.load('../models/toxic_comment_model.pt')
    print("\nMejor modelo guardado:")
    print(f"Accuracy de validación: {checkpoint['val_accuracy']:.4f}")
    print(f"Accuracy de entrenamiento: {checkpoint['train_accuracy']:.4f}")
    print(f"Overfitting: {checkpoint['overfitting']:.2f}%")
except FileNotFoundError:
    print("No se encontró ningún modelo con overfitting < 5%")
# Calcular el porcentaje de overfitting
overfitting_percentage = abs(train_accuracy - val_accuracy) * 100

print(f"\nMejor accuracy del modelo: {best_val_accuracy:.4f}")

# Resultados finales
print("\nResultados finales:")
print(f"Mejor Accuracy: {best_val_accuracy:.4f}")
print(f"Loss final: {val_loss:.4f}")
print(f"Mejor Accuracy de entrenamiento: {best_train_accuracy:.4f}")
print(f"Mejor Accuracy de validación: {best_val_accuracy:.4f}")
print(f"Porcentaje de overfitting: {overfitting_percentage:.2f}%")

# Evaluación del overfitting
if overfitting_percentage > 10:
    print("WARNING: Alto nivel de overfitting detectado!")
elif overfitting_percentage > 5:
    print("AVISO: Nivel moderado de overfitting")
else:
    print("BIEN: Nivel de overfitting aceptable")



Using device: cpu

Distribución de clases:
IsToxic
0    538
1    462
Name: count, dtype: int64

Tamaño del conjunto de entrenamiento: 800
Tamaño del conjunto de validación: 200

Distribución de clases en entrenamiento:
IsToxic
0    430
1    370
Name: count, dtype: int64

Distribución de clases en validación:
IsToxic
0    108
1     92
Name: count, dtype: int64
Batch size ajustado: 16


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\vitta\Desktop\F5_Bootcamp\G2_Youtube\.venv\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/8


Training: 100%|██████████| 50/50 [02:00<00:00,  2.41s/it, loss=0.6538, accuracy=0.5238]


Train Loss: 0.7030, Train Accuracy: 0.5238
Validation Loss: 0.6791, Validation Accuracy: 0.5750
No se guardó el modelo - Overfitting (5.12%) > 5%

Epoch 2/8


...

Train Loss: 0.4505, Train Accuracy: 0.7950
Validation Loss: 0.4853, Validation Accuracy: 0.7950

Guardado nuevo mejor modelo:
Accuracy de validación: 0.7950
Accuracy de entrenamiento: 0.7950
Overfitting: 0.00%

Epoch 8/8


Training: 100%|██████████| 50/50 [01:57<00:00,  2.35s/it, loss=0.4560, accuracy=0.8337]


Train Loss: 0.3958, Train Accuracy: 0.8337
Validation Loss: 0.4877, Validation Accuracy: 0.7550


C:\Users\vitta\AppData\Local\Temp\ipykernel_24552\906905357.py:180: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../models/toxic_comment_model.pt')


Mejor modelo guardado:
Accuracy de validación: 0.7950
Accuracy de entrenamiento: 0.7950
Overfitting: 0.00%

Mejor accuracy del modelo: 0.7950

Resultados finales:
Mejor Accuracy: 0.7950
Loss final: 0.4877
Mejor Accuracy de entrenamiento: 0.7950
Mejor Accuracy de validación: 0.7950
Porcentaje de overfitting: 7.87%
AVISO: Nivel moderado de overfitting


In [7]:
# Cargar el modelo guardado
try:
    checkpoint = torch.load('../models/toxic_comment_model.pt')
    model.load_state_dict(checkpoint['model_state_dict'])
    print("\nModelo cargado exitosamente")
    print(f"Accuracy de validación: {checkpoint['val_accuracy']:.4f}")
    print(f"Accuracy de entrenamiento: {checkpoint['train_accuracy']:.4f}")
    print(f"Overfitting: {checkpoint['overfitting']:.2f}%")
    
    # Guardar solo el tokenizer
    tokenizer.save_pretrained('../models/toxic_comment_tokenizer')
    print("Tokenizer guardado exitosamente")
    
except Exception as e:
    print(f"Error: {e}")

# Verificar que el tokenizer se guardó correctamente
if os.path.exists('../models/toxic_comment_tokenizer'):
    print("Tokenizer verificado en: ../models/toxic_comment_tokenizer")

C:\Users\vitta\AppData\Local\Temp\ipykernel_24552\406936604.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../models/toxic_comment_model.pt')



Modelo cargado exitosamente
Accuracy de validación: 0.7950
Accuracy de entrenamiento: 0.7950
Overfitting: 0.00%
Tokenizer guardado exitosamente
Tokenizer verificado en: ../models/toxic_comment_tokenizer


In [9]:
# Cargar el checkpoint para ver los resultados
checkpoint = torch.load('../models/toxic_comment_model.pt')

print("\nResultados para copiar al README:")
print(f"Accuracy de validación: {checkpoint['val_accuracy']:.4f}")
print(f"Accuracy de entrenamiento: {checkpoint['train_accuracy']:.4f}")
print(f"Overfitting: {checkpoint['overfitting']:.2f}%")

C:\Users\vitta\AppData\Local\Temp\ipykernel_24552\1274574373.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../models/toxic_comment_model.pt')



Resultados para copiar al README:
Accuracy de validación: 0.7950
Accuracy de entrenamiento: 0.7950
Overfitting: 0.00%


In [14]:
import os

# Crear el directorio si no existe
if not os.path.exists('../model-huggin'):
    os.makedirs('../model-huggin')

try:
    # Cargar el mejor modelo guardado anteriormente
    checkpoint = torch.load('../models/toxic_comment_model.pt')

    # Cargar el estado del modelo
    model.load_state_dict(checkpoint['model_state_dict'])
    print("\nMejor modelo cargado con:")
    print(f"- Accuracy de validación: {checkpoint['val_accuracy']:.4f}")
    print(f"- Accuracy de entrenamiento: {checkpoint['train_accuracy']:.4f}")
    print(f"- Overfitting: {checkpoint['overfitting']:.2f}%")


    # Guardar en formato Hugging Face
    model.save_pretrained('../model-huggin')
    tokenizer.save_pretrained('../model-huggin')
    
    # Verificar los archivos guardados
    files = os.listdir('../model-huggin')
    print("\nArchivos guardados en model-huggin:")
    for file in files:
        print(f"- {file}")
    
    print("\n✅ Modelo y tokenizer guardados correctamente en formato Hugging Face")
    print("Ahora puedes subir estos archivos a tu repositorio de Hugging Face")
except Exception as e:
    print(f"Error: {e}")

C:\Users\vitta\AppData\Local\Temp\ipykernel_24552\167465444.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../models/toxic_comment_model.pt')



Mejor modelo cargado con:
- Accuracy de validación: 0.7950
- Accuracy de entrenamiento: 0.7950
- Overfitting: 0.00%

Archivos guardados en model-huggin:
- config.json
- model.safetensors
- special_tokens_map.json
- tokenizer_config.json
- vocab.txt

✅ Modelo y tokenizer guardados correctamente en formato Hugging Face
Ahora puedes subir estos archivos a tu repositorio de Hugging Face
